In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import sys
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
csvs = [] 
for dirname, _, filenames in os.walk('./archive'):
    for filename in filenames:
        path = os.path.join(dirname, filename) 
        print(path)
        csvs.append( pd.read_csv(path) )
        
df = pd.concat(csvs, axis = 0)

./archive/2020.csv
./archive/2021.csv
./archive/2019.csv
./archive/2018.csv
./archive/2015.csv
./archive/2014.csv
./archive/2016.csv
./archive/2017.csv
./archive/2013.csv
./archive/2012.csv
./archive/2010.csv
./archive/2011.csv


In [3]:
df.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,0,1343644462036086785,1343320495127633920,1.609185e+12,2020-12-28 19:46:18,0,NaN,Entertainment will be critical when cars drive...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1,1343619610617077760,1343386617294295040,1.609179e+12,2020-12-28 18:07:33,0,NaN,@kimpaquette Just meeting with Larry Ellison t...,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'kimpaquette', 'name': 'Kim P...",NaN,NaN,NaN,NaN
2,2,1343608616960491521,1343576442722893825,1.609176e+12,2020-12-28 17:23:51,0,NaN,@richierichhhhh_ Absolutely,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'richierichhhhh_', 'name': 'R...",NaN,NaN,NaN,NaN
3,3,1343608530998153222,1343320495127633920,1.609176e+12,2020-12-28 17:23:31,0,NaN,What should Tesla do with in-car gaming in an ...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,4,1343431408052662273,1343043963096326147,1.609134e+12,2020-12-28 05:39:42,0,NaN,@PPathole @WSJ Absolutely,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'PPathole', 'name': 'Pranay P...",NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43074 entries, 0 to 26
Data columns (total 39 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       43074 non-null  int64  
 1   id               43074 non-null  int64  
 2   conversation_id  43074 non-null  int64  
 3   created_at       43074 non-null  float64
 4   date             43074 non-null  object 
 5   timezone         43074 non-null  int64  
 6   place            0 non-null      float64
 7   tweet            43074 non-null  object 
 8   language         43074 non-null  object 
 9   hashtags         43074 non-null  object 
 10  cashtags         43074 non-null  object 
 11  user_id          43074 non-null  int64  
 12  user_id_str      43074 non-null  int64  
 13  username         43074 non-null  object 
 14  name             43074 non-null  object 
 15  day              43074 non-null  int64  
 16  hour             43074 non-null  int64  
 17  link           

**Question 1: Estimate the average number of people that 'like' any given tweet by Elon Musk.**

In [5]:
q1 = round( df.nlikes.mean() )
print(q1)

14705


**Question 2: From the above, estimate how many people view one of his tweets, on average. Assume that engagement rate, calculated by summing likes, replies, and retweets, and dividing by views, is roughly .05 for any given tweet.**

In [6]:
q2 = round( ( df.nlikes.mean() + df.nreplies.mean() + df.nretweets.mean() ) / 0.05 )
print(q2)

338864


**Question 3: Estimate the average amount of data (in megabytes) stored for each Elon Musk tweet. Consider the sizes and types of any attached media.**

In [7]:
import requests

cols = ['date', 'tweet', 'language', 'hashtags', 'cashtags', 'username', 'name', 'link', 'urls', 'photos', 'thumbnail', 'quote_url', 'search', 'reply_to']

base_bits = 1 + 64 * 12 + 64 * 12
estimate_video_bits = 200 * 1024 * 1024
total_bits = 0

n_sample = 250
sdf = df.sample(n=n_sample, random_state=498)
for idx, row in sdf.iterrows():
    total_bits += base_bits
    for col in cols:
        total_bits += sys.getsizeof(row[col]) * 8
    
    if row['photos'] and len(row['photos']) > 2:
#         print(row['photos'])
        photos = row['photos'].replace("'", "")
        photos = photos.replace("[", "")
        photos = photos.replace("]", "")
        photos = photos.split(',')
        for photo in photos:
            photo = photo.strip()
#             print(photo)
            try:
                resp = requests.head(photo, allow_redirects=True)
                total_bits += int(resp.headers['Content-Length']) * 8
            except:
                pass

    if isinstance(row['thumbnail'], str):
        thumbnail = row['thumbnail'].strip()
#         print(thumbnail)
        if "video" in thumbnail:
#             print(thumbnail)
            total_bits += estimate_video_bits
        try:
            resp = requests.head(thumbnail, allow_redirects=True)
            total_bits += int(resp.headers['Content-Length']) * 8
        except:
            pass


q3 = round(total_bits / 8 / n_sample)
print(q3)

119110


**Question 4: Using the answers above, estimate the total data transfer involved in displaying any given Elon Musk tweet to Twitter users.**

In [8]:
q4 = q2 * q3
print(q4)

40362091040


**Question 5: Estimate how many Elon tweets are viewed, per- minute, by the site's userbase.**

In [9]:
q5 = round( q2 * df.shape[0] / (11 * 365 * 24 * 60) )
print(q5)

2525


**Question 6: Using the above, estimate how much data is accessed per-minute by Twitter in the process of displaying Elon Musk tweets.**

In [10]:
q6 = q3 * q5
print(q6)

300752750


**Question 7: Choose a function that involves modifying a tweet (say, updating its 'likes' count, which you can assume is cached in a tweet's associated database row). How much data do you believe is touched by this operation on a monthly basis? Consider the amount of data affected, the frequency at which the operation is carried out, and so on.**

I'm going to estimate the function of updating 'retweet' count for any given Elon Musk tweet.

In [11]:
# First, estimate the average retweets per tweet.
avgRetweetPerTweet = round( df.nretweets.mean() )

# Second, convert the previous result into monthly retweet count.
monthlyRetweetCnt = (avgRetweetPerTweet * df.shape[0] / (11 * 12) )

# Third, calculate the data touched by updating the column "nretweets".
# I assume that every user can retweet the same tweet multiple times,
# so I do not need to check if the user has already retweeted this tweet or not.
# Also, I'll have to retrieve the original value first and then store the value back again,
# therefore, one update on "nretweets" would require 2 * 8 Bytes of data,
# assuming the database uses int64 to store the value.

q7 = round( monthlyRetweetCnt * 2 * 8 )
print(q7)

9345753
